In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# get excel file in a xls variable
xls = pd.ExcelFile("../Data/Folkhalsomyndigheten_Covid19_Vaccine.xlsx")


In [2]:
vaccinationer_tidsserie = pd.read_excel(xls, 'Vaccinationer tidsserie')
vaccinerade_tidsserie = pd.read_excel(xls, 'Vaccinerade tidsserie')
vaccinerade_dos_3 = pd.read_excel(xls, 'Vaccinerade tidsserie dos 3')
vaccinerade_dos_4 = pd.read_excel(xls, 'Vaccinerade tidsserie dos 4')
vaccinerade_dos_5 = pd.read_excel(xls, 'Vaccinerade tidsserie dos 5')
dos_1_to_3_per_age_group = pd.read_excel(xls, 'Dos 1 till 3 per åldersgrupp')
dos_4_per_age_group = pd.read_excel(xls, 'Dos 4 per åldersgrupp')
dos_4_18_plus = pd.read_excel(xls, 'Dos 4 18+')
dos_5_per_age_group  = pd.read_excel(xls, 'Dos 5 per åldersgrupp')
vaccinerade_gender  = pd.read_excel(xls, 'Vaccinerade kön')
vaccinerade_kommun  = pd.read_excel(xls, 'Vaccinerade kommun')
vaccinerade_kommun_dos_3  = pd.read_excel(xls, 'Vaccinerade kommun dos 3')
vaccinerade_kommun_dos_4  = pd.read_excel(xls, 'Vaccinerade kommun dos 4')
vaccinerade_kommun_dos_5  = pd.read_excel(xls, 'Vaccinerade kommun dos 5')
vaccinerade_kommun_och_ålder  = pd.read_excel(xls, 'Vaccinerade kommun och ålder')



In [3]:
vaccinationer_tidsserie.head()

,Vecka,År,Region,Antal vaccinationer
0,52,2020,| Sverige |,2478
1,52,2020,Stockholm,289
2,52,2020,Uppsala,9
3,52,2020,Södermanland,116
4,52,2020,Östergötland,214


In [4]:
vaccinerade_tidsserie['Region'].value_counts

<bound method IndexOpsMixin.value_counts of 0        | Sverige |
1        | Sverige |
2          Stockholm
3          Stockholm
4            Uppsala
            ...     
4219        Jämtland
4220    Västerbotten
4221    Västerbotten
4222      Norrbotten
4223      Norrbotten
Name: Region, Length: 4224, dtype: object>

In [5]:
vaccinerade_kommun_och_ålder.head(3)

,Län,Län_namn,Kommun,Kommun_namn,Ålder,Befolkning,Antal minst 1 dos,Antal minst 2 doser,Antal 3 doser,Antal 4 doser,Andel minst 1 dos,Andel minst 2 doser,Andel 3 doser,Andel 4 doser
0,1,Stockholms län,114,Upplands Väsby,12-15,2422,1206,1046,NaN,NaN,0.497936,0.431874,NaN,NaN
1,1,Stockholms län,114,Upplands Väsby,16-17,1203,839,755,NaN,NaN,0.697423,0.627598,NaN,NaN
2,1,Stockholms län,114,Upplands Väsby,18-29,6692,4887,4469,1959.0,NaN,0.730275,0.667812,0.292738,NaN


## 2.a

In [6]:
#nunique counts all the unique values from a datframe column

antal_län = vaccinerade_kommun_och_ålder['Län'].nunique()
print(f"Antal län i sverige = {antal_län}")

Antal län i sverige = 21


## 2.b

In [7]:
antal_Kommuner = vaccinerade_kommun_och_ålder['Kommun'].nunique()
print(f"Antal Kommuner i sverige = {antal_Kommuner}")

Antal Kommuner i sverige = 290


# 2.c

In [8]:
total_population = 10_502_959
print(f"Total befolkning i sverige = {total_population}")

Total befolkning i sverige = 10502959


## 2. d)

In [9]:
# By splitting the age-group into more maneageable int
# by taking the Ålder column, and takin the indices from 0 to 2 not included, ie: indices 0 and 1
# then converting it to an int
vaccinerade_kommun_och_ålder["Ålder"] =  vaccinerade_kommun_och_ålder["Ålder"].str[:2].astype(int)


In [10]:
import numpy as np
#inspo from: 
#https://stackoverflow.com/questions/28236305/how-do-i-sum-values-in-a-column-that-match-a-given-condition-using-pandas

#Essentially this line uses the numpy method -> where, in which it takes in the condition which is: vaccinerade_kommun_och_ålder["Ålder"] >= 18
#then  looks the values in vaccinerade_kommun_och_ålder['Befolkning'],if the condition is met 
# and the 0 is axis 0 which is rows 
# Then calculates the sum of all these values that meet the condition
over_17 = np.where(vaccinerade_kommun_och_ålder["Ålder"] >= 18, vaccinerade_kommun_och_ålder['Befolkning'], 0).sum()

In [11]:
under_18 = total_population - over_17
under_18
f'Number of people under 18 = {under_18:,}'

'Number of people under 18 = 2,155,539'

# 2.e)

In [12]:

vaccinerade_kommun = pd.read_excel(xls, 'Vaccinerade kommun och ålder')

# The filter function will return just the columns that i need for the plot

age_distribution = vaccinerade_kommun.filter(['Ålder','Befolkning'])

age_distribution.head(2)

,Ålder,Befolkning
0,12-15,2422
1,16-17,1203


In [13]:
#Group all similar age groups and calculate their values using the groupby method
# Use the agg. function to pick which columns will be return in the groupby then calculatin the sum() of all values of every row

age_distribution = age_distribution.groupby('Ålder').agg({'Befolkning':'sum'}).reset_index()

In [14]:
# Since under 12 year olds arent represented in the data, under_12 is just total population minus the population represented in the data 
over_12 = age_distribution['Befolkning'].sum()
under_12 = total_population - over_12
under_12

1410169

In [15]:
age_distribution.loc[len(age_distribution)] = ['0-11',under_12]
age_distribution = age_distribution

In [16]:
age_distribution_sorted = age_distribution.sort_values(by='Ålder')
age_distribution_sorted.head(2)

,Ålder,Befolkning
10,0-11,1410169
0,12-15,503831


In [17]:
import plotly_express as px
import matplotlib.pyplot as plt

fig = px.bar(age_distribution_sorted, x="Ålder", y='Befolkning')
fig.show()

## 2. f)

### minst dos 1

In [18]:
import plotly_express as px
import matplotlib.pyplot as plt

minst_dos1 = vaccinerade_kommun_och_ålder.groupby('Län_namn')['Andel minst 1 dos'].mean() 
fig = px.bar(x=minst_dos1.index, y=minst_dos1, title="Mean Andel med minst 1 dos")
fig.update_yaxes(title="Andelen")
fig.update_xaxes(title="Län")
fig.show()



### minst dos 2

In [19]:
import plotly_express as px
import matplotlib.pyplot as plt

minst_dos2 = vaccinerade_kommun_och_ålder.groupby('Län_namn')['Andel minst 2 doser'].mean()

fig = px.bar(x=minst_dos2.index, y=minst_dos2, title="Andel minst 2 doser")
fig.update_layout(title="Andel dos 2")
fig.update_yaxes(title="Andelen")
fig.update_xaxes(title="Län")
fig.show()

### minst 3 doser

In [20]:
import plotly_express as px
import matplotlib.pyplot as plt

minst_dos3 = vaccinerade_kommun_och_ålder.groupby('Län_namn')['Andel 3 doser'].mean()

fig = px.bar(x=minst_dos3.index, y=minst_dos3)
fig.update_layout(title="Andel dos 3")
fig.update_yaxes(title="Andelen",showgrid=False,showticklabels=True)
fig.update_xaxes(title="Län", showgrid=False, zeroline=True,showticklabels=True, showline=False)
fig.show()



In [21]:
dff = vaccinerade_kommun_och_ålder.groupby('Län_namn').agg({'Andel minst 1 dos':'mean', 'Andel minst 2 doser':'mean', 'Andel 3 doser':'mean', 'Andel 4 doser':'mean'})

In [22]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

columns = ['Andel minst 1 dos', 'Andel minst 2 doser','Andel 3 doser', 'Andel 4 doser']
fig = make_subplots(rows=1, cols=1, subplot_titles=["Mean andel dos per Län"])
for col in columns:
    fig.add_trace(
        go.Bar(x=dff.index, y=dff[col],
        name=col, showlegend=True)
            
    )
fig.update_layout(showlegend=True)
fig.update_yaxes(range=[0,1], title="mean andel dos")
fig.update_xaxes(title="Län")
fig.show()

In [23]:
stockholm_län =vaccinerade_kommun_och_ålder[vaccinerade_kommun_och_ålder["Län_namn"] == "Stockholms län"]

In [24]:
västra_götaland_län =vaccinerade_kommun_och_ålder[vaccinerade_kommun_och_ålder["Län_namn"] == "Västra Götalands län"]

In [25]:
västra_götaland_län['Andel 3 doser'].fillna(0, inplace=True)

C:\Users\CYRILLE\AppData\Local\Temp\ipykernel_6456\4057759334.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [26]:
västra_götaland_län['Andel 4 doser'].fillna(0, inplace=True)

C:\Users\CYRILLE\AppData\Local\Temp\ipykernel_6456\4203142793.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [27]:
stockholm_län['Andel 3 doser'].fillna(0, inplace=True)

C:\Users\CYRILLE\AppData\Local\Temp\ipykernel_6456\71766071.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [28]:
stockholm_län['Andel 4 doser'].fillna(0, inplace=True)

C:\Users\CYRILLE\AppData\Local\Temp\ipykernel_6456\1147220974.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [29]:
the_big_2 = pd.concat([västra_götaland_län, stockholm_län])


In [30]:
df = the_big_2.groupby('Län_namn').agg({'Andel minst 1 dos':'mean', 'Andel minst 2 doser':'mean', 'Andel 3 doser':'mean', 'Andel 4 doser':'mean'}).T

In [31]:
df.index

Index(['Andel minst 1 dos', 'Andel minst 2 doser', 'Andel 3 doser',
       'Andel 4 doser'],
      dtype='object')

In [32]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot(df: pd.DataFrame):
    columns = ['Stockholms län', 'Västra Götalands län']
    fig = make_subplots(rows=1, cols=1, subplot_titles=["Mean andel dos per region"])
    for i, col in enumerate(columns):
        fig.add_trace(
            go.Bar(x=df.index, y=df[col],
            name=col, showlegend=True)
            
        )
    fig.update_layout(showlegend=True)
    fig.update_yaxes(range=[0,1], title="mean andel dos")
    fig.update_xaxes(title="mean andel dos per län")
    fig.show()

In [33]:
plot(df)